In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import statsmodels.api as sm

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)

plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings
do_compile = False
threads = 16


#dowload nlopt
import consav
import Estimate
consav.cpptools.setup_nlopt()

# Solve the model

In [2]:
# compile c++ files
T = 5
specs = {
    'model 3':{'latexname':'$\sigma_{\psi}=0.1$', 'par':{'sigma_love':0.1, 'sigma_K': 1.0, 'T': T , 'threads':threads}},
}


# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    print(f'{name} loading...',end='')
    
    # setup model
    models[name] = LimitedCommitmentModelClass(name=name,par=spec['par'])
    if do_compile & (m==0):
        compile_now = True
    else:
        compile_now = False
    models[name].link_to_cpp(force_compile=compile_now)
    models[name].spec = spec
    
    print(' solving...')
    %time models[name].solve()

    

model 3 loading... solving...
CPU times: total: 39min 56s
Wall time: 6min 49s


# Estimate the model

In [91]:
#Estimate the model one time
model = models['model 3']

model.sim.init_A[:] = 8.0 # temp just to check that it works, if they have too little initial saving they will just consume everything, and saving is therefore close to zero, and the model cannot be estimated

np.random.seed(model.par.seed)
data = Estimate.create_data(model,start_p = 2, end_p = 4, to_xl = False)
data_reg = Estimate.aux_est(data,print_reg=True)
data_reg, Wald_FC, Wald_NC = Estimate.main_est(data_reg,print_reg=True) 

c:\Users\czk481\Anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Residuals from hours equation
                                 OLS Regression Results                                
Dep. Variable:        delta_log_Labor   R-squared (uncentered):                   0.010
Model:                            OLS   Adj. R-squared (uncentered):              0.010
Method:                 Least Squares   F-statistic:                              627.8
Date:                Wed, 17 Jan 2024   Prob (F-statistic):                        0.00
Time:                        20:24:51   Log-Likelihood:                         -22709.
No. Observations:              187857   AIC:                                  4.542e+04
Df Residuals:                  187854   BIC:                                  4.546e+04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----

c:\Users\czk481\Anaconda3\lib\site-packages\statsmodels\base\model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
